## PS: All the data exploration phase is the same as in essai 1 . This essai2 is the final one.

In [1]:
import pandas as pd
import re
from scipy import stats
from sklearn.pipeline import make_pipeline
from sklearn.kernel_ridge import KernelRidge
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder, Imputer
from scipy.stats import skew
from sklearn.linear_model import Lasso
import xgboost as xgb
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from random import shuffle
from sklearn.svm import SVC
from sklearn.linear_model import ElasticNetCV
from sklearn.datasets import make_regression
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn import svm
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold 
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from pprint import pprint
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import label_binarize
from sklearn import metrics
from sklearn import linear_model
import seaborn as sns
from sklearn.svm import SVC
from time import time
import numpy as np
import seaborn as se
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv', sep =',', error_bad_lines = False)
test = pd.read_csv('test.csv', sep =',', error_bad_lines = False)
testid= pd.read_csv('test.csv', sep =',', error_bad_lines = False)
full = train.append(test, ignore_index=True)
full.head()


C:\Users\khalid\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


1stFlrSF  2ndFlrSF  3SsnPorch Alley  BedroomAbvGr BldgType BsmtCond  \
0       856       854          0   NaN             3     1Fam       TA   
1      1262         0          0   NaN             3     1Fam       TA   
2       920       866          0   NaN             3     1Fam       TA   
3       961       756          0   NaN             3     1Fam       Gd   
4      1145      1053          0   NaN             4     1Fam       TA   

  BsmtExposure  BsmtFinSF1  BsmtFinSF2  ...   SaleType ScreenPorch  Street  \
0           No       706.0         0.0  ...         WD           0    Pave   
1           Gd       978.0         0.0  ...         WD           0    Pave   
2           Mn       486.0         0.0  ...         WD           0    Pave   
3           No       216.0         0.0  ...         WD           0    Pave   
4           Av       655.0         0.0  ...         WD           0    Pave   

   TotRmsAbvGrd TotalBsmtSF  Utilities WoodDeckSF YearBuilt YearRemodAdd  \
0             8       856.0     AllPub          0      2003         2003   
1             6      1262.0     AllPub        298      1976         1976   
2             6       920.0     AllPub          0      2001         2002   
3             7       756.0     AllPub          0      1915         1970   
4             9      1145.0     AllPub        192      2000         2000   

  YrSold  
0   2008  
1   2007  
2   2008  
3   2006  
4   2008  

[5 rows x 81 columns]

In [3]:
#remove outliers ( based on the previous essai)
train = train[train.GrLivArea < 4500]
train.shape

(1458, 81)

In [4]:
#recheck for mission data
missing_values = pd.concat([train.isnull().sum(), test.isnull().sum()], axis=1, keys=['Train', 'Test'])
missing_values[missing_values.sum(axis=1) > 0]

C:\Users\khalid\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Train    Test
Alley          1367  1352.0
BsmtCond         37    45.0
BsmtExposure     38    44.0
BsmtFinSF1        0     1.0
BsmtFinSF2        0     1.0
BsmtFinType1     37    42.0
BsmtFinType2     38    42.0
BsmtFullBath      0     2.0
BsmtHalfBath      0     2.0
BsmtQual         37    44.0
BsmtUnfSF         0     1.0
Electrical        1     0.0
Exterior1st       0     1.0
Exterior2nd       0     1.0
Fence          1177  1169.0
FireplaceQu     690   730.0
Functional        0     2.0
GarageArea        0     1.0
GarageCars        0     1.0
GarageCond       81    78.0
GarageFinish     81    78.0
GarageQual       81    78.0
GarageType       81    76.0
GarageYrBlt      81    78.0
KitchenQual       0     1.0
LotFrontage     259   227.0
MSZoning          0     4.0
MasVnrArea        8    15.0
MasVnrType        8    16.0
MiscFeature    1404  1408.0
PoolQC         1452  1456.0
SaleType          0     1.0
TotalBsmtSF       0     1.0
Utilities         0     2.0

In [5]:
# Spliting to columns that describes the features and outcome
y = train.pop('SalePrice')
data = pd.concat([train, test], keys=['train', 'test'])

In [6]:
test.shape, train.shape , data.shape

((1459, 80), (1458, 80), (2917, 80))

In [7]:
# Deleting columns that are more than 50% missing
data.drop(['PoolQC', 'MiscFeature', 'FireplaceQu', 'Fence', 'Alley'],
              axis=1, inplace=True)
data.shape

(2917, 75)

In [8]:
#filling with most popular values
data['MSZoning'] = data['MSZoning'].fillna(data['MSZoning'].mode()[0])
data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])
data['MasVnrType'] = data['MasVnrType'].fillna(data['MasVnrType'].mode()[0])
data['Functional'] = data['Functional'].fillna(data['Functional'].mode()[0])
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])
data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])
data['Utilities'] = data['Utilities'].fillna(data['Utilities'].mode()[0])

# LotFrontage  NA in all. I suppose NA means 0
data['LotFrontage'] = data['LotFrontage'].fillna(data['LotFrontage'].mean())
data['MasVnrArea'] = data['MasVnrArea'].fillna(data['MasVnrArea'].mean())
data['BsmtFullBath'] = data['BsmtFullBath'].fillna(data['BsmtFullBath'].median())
data['BsmtHalfBath'] = data['BsmtHalfBath'].fillna(data['BsmtHalfBath'].median())
data['BsmtUnfSF'] = data['BsmtUnfSF'].fillna(data['BsmtUnfSF'].mean())

for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    data[col] = data[col].fillna('NoBSMT')
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageQual', 'GarageCond'):
    data[col] = data[col].fillna('NoGRG')

data['BsmtFinSF1'] = data['BsmtFinSF1'].fillna(0)   
data['BsmtFinSF2'] = data['BsmtFinSF2'].fillna(0)  
data['TotalBsmtSF'] = data['TotalBsmtSF'].fillna(0)
data['GarageArea'] = data['GarageArea'].fillna(0.0)
data['GarageCars'] = data['GarageCars'].fillna(0.0)

#Create anew column
data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']
data.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GarageYrBlt'], axis=1, inplace=True)


In [9]:
data.shape

(2917, 72)

In [10]:
#Log transformation of SalePrice to get a normal distribution
y= np.log(y)

In [11]:
#convert the numerical varible to categorical
data.MSSubClass.dtypes

dtype('int64')

In [12]:
def num2cat(x):
    return str(x)
data['MSSubClass_str'] = data['MSSubClass'].apply(num2cat)
data.pop('MSSubClass')


train  0        60
       1        20
       2        60
       3        70
       4        60
       5        50
       6        20
       7        60
       8        50
       9       190
       10       20
       11       60
       12       20
       13       20
       14       20
       15       45
       16       20
       17       90
       18       20
       19       20
       20       60
       21       45
       22       20
       23      120
       24       20
       25       20
       26       20
       27       20
       28       20
       29       30
              ... 
test   1429     30
       1430     50
       1431     30
       1432    190
       1433     50
       1434    120
       1435    120
       1436     20
       1437     90
       1438     20
       1439     80
       1440     20
       1441     20
       1442     20
       1443     20
       1444     20
       1445     90
       1446    160
       1447     20
       1448     90
       1449    180
       1450 

In [13]:
#Convert categorical columns to binary using one-hot encoding
for col in data.dtypes[data.dtypes == 'object'].index:
    for_dummy = data.pop(col)
    data = pd.concat([data, pd.get_dummies(for_dummy, prefix=col)], axis=1)

In [14]:
 data.tail()

Id  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
test 1454  2915         21.0     1936            4            7       1970   
     1455  2916         21.0     1894            4            5       1970   
     1456  2917        160.0    20000            5            7       1960   
     1457  2918         62.0    10441            5            5       1992   
     1458  2919         74.0     9627            7            5       1993   

           YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  \
test 1454          1970         0.0         0.0         0.0   
     1455          1970         0.0       252.0         0.0   
     1456          1996         0.0      1224.0         0.0   
     1457          1992         0.0       337.0         0.0   
     1458          1994        94.0       758.0         0.0   

                 ...          MSSubClass_str_30  MSSubClass_str_40  \
test 1454        ...                          0                  0   
     1455        ...                          0                  0   
     1456        ...                          0                  0   
     1457        ...                          0                  0   
     1458        ...                          0                  0   

           MSSubClass_str_45  MSSubClass_str_50  MSSubClass_str_60  \
test 1454                  0                  0                  0   
     1455                  0                  0                  0   
     1456                  0                  0                  0   
     1457                  0                  0                  0   
     1458                  0                  0                  1   

           MSSubClass_str_70  MSSubClass_str_75  MSSubClass_str_80  \
test 1454                  0                  0                  0   
     1455                  0                  0                  0   
     1456                  0                  0                  0   
     1457                  0                  0                  0   
     1458                  0                  0                  0   

           MSSubClass_str_85  MSSubClass_str_90  
test 1454                  0                  0  
     1455                  0                  0  
     1456                  0                  0  
     1457                  1                  0  
     1458                  0                  0  

[5 rows x 291 columns]

In [15]:
### Splitting the data
train = data.loc['train'].drop('Id', axis=1).select_dtypes(include=[np.number]).values
test = data.loc['test'].drop('Id', axis=1).select_dtypes(include=[np.number]).values

In [16]:
train.shape

(1458, 290)

In [17]:
#training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train,y,test_size=0.1,random_state=200)

In [18]:
# Prints R2 and RMSE scores
def get_score(prediction, lables):    
    print('R2: {}'.format(r2_score(prediction, lables)))
    print('RMSE: {}'.format(np.sqrt(mean_squared_error(prediction, lables))))

# Shows scores for train and validation sets    
def train_test(estimator, x_trn, x_tst, y_trn, y_tst):
    prediction_train = estimator.predict(x_trn)
    # Printing estimator
    print(estimator)
    # Printing train scores
    get_score(prediction_train, y_trn)
    prediction_test = estimator.predict(x_tst)
    # Printing test scores
    print("Test")
    get_score(prediction_test, y_tst)

## All the models below went to a GridSerachCV to find the best estimator (check essai1)

## GradientBoostingRegressor

In [19]:
GBR = GradientBoostingRegressor(n_estimators=12000,
            learning_rate=0.05, max_depth=3, max_features='sqrt',
            min_samples_leaf=15, min_samples_split=10, loss='huber')
GBR.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=12000,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [20]:
train_test(GBR, x_train, x_test, y_train, y_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=12000,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)
R2: 0.9866907534039374
RMSE: 0.04518740720831715
Test
R2: 0.924329902415055
RMSE: 0.0979976191032606


In [21]:
# Average R2 score and standart deviation of 5-fold cross-validation
scores = cross_val_score(GBR, train, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.91 (+/- 0.01)


## Lasso

In [22]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
lasso.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))])

In [23]:
train_test(lasso, x_train, x_test, y_train, y_test)

Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))])
R2: 0.9327242241294451
RMSE: 0.09988410073894072
Test
R2: 0.9289647331217882
RMSE: 0.09421505275801738


In [24]:
# Average R2 score and standart deviation of 5-fold cross-validation
scores = cross_val_score(lasso, train, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.92 (+/- 0.01)


## ElasticNetCV

In [25]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
ENet.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('elasticnet', ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=3, selection='cyclic', tol=0.0001, warm_start=False))])

In [26]:
train_test(ENet, x_train, x_test, y_train, y_test)

Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('elasticnet', ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=3, selection='cyclic', tol=0.0001, warm_start=False))])
R2: 0.9338990489977147
RMSE: 0.0990905373419731
Test
R2: 0.9287968056137614
RMSE: 0.09444584620484608


In [27]:
# Average R2 score and standart deviation of 5-fold cross-validation
scores = cross_val_score(ENet, train, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.92 (+/- 0.01)


## LGBMRegressor

In [28]:
m_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
m_lgb.fit(x_train,y_train)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
       boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       feature_fraction=0.2319, feature_fraction_seed=9,
       importance_type='split', learning_rate=0.05, max_bin=55,
       max_depth=-1, min_child_samples=20, min_child_weight=0.001,
       min_data_in_leaf=6, min_split_gain=0.0, min_sum_hessian_in_leaf=11,
       n_estimators=720, n_jobs=-1, num_leaves=5, objective='regression',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [29]:
train_test(m_lgb, x_train, x_test, y_train, y_test)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
       boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       feature_fraction=0.2319, feature_fraction_seed=9,
       importance_type='split', learning_rate=0.05, max_bin=55,
       max_depth=-1, min_child_samples=20, min_child_weight=0.001,
       min_data_in_leaf=6, min_split_gain=0.0, min_sum_hessian_in_leaf=11,
       n_estimators=720, n_jobs=-1, num_leaves=5, objective='regression',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
R2: 0.9648858252837197
RMSE: 0.07293611726596502
Test
R2: 0.9320707100293508
RMSE: 0.09412367057939823


In [30]:
# Average R2 score and standart deviation of 5-fold cross-validation
scores = cross_val_score(m_lgb, train, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.92 (+/- 0.01)


## XGBRegressor

In [ ]:
m_xgb = xgb.XGBRegressor(colsample_bytree=0.2,
                       gamma=0.0,
                       learning_rate=0.05,
                       max_depth=6,
                       min_child_weight=1.5,
                       n_estimators=7200,
                       reg_alpha=0.9,
                       reg_lambda=0.6,
                       subsample=0.2,
                       seed=42,
                       silent=1)
m_xgb.fit(x_train,y_train)

In [ ]:
train_test(m_xgb, x_train, x_test, y_train, y_test)

In [ ]:
# Average R2 score and standart deviation of 5-fold cross-validation
scores = cross_val_score(m_xgb, train, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## SVR (the model didn't get better while adding SVR)

In [ ]:
svr=SVR(C=13, cache_size=200, coef0=0.0, degree=3, epsilon=0.03, gamma=0.0005,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
svr.fit(x_train,y_train)

In [ ]:
train_test(svr, x_train, x_test, y_train, y_test)

In [ ]:
# Average R2 score and standart deviation of 5-fold cross-validation
scores = cross_val_score(svr, train, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Combining Models (Avereging)

## I choose avering because it gives better results than stacking (essai1)

In [31]:
def averagingModels(X, train, labels, models=[]):
    for model in models:
        model.fit(train, labels)
    predictions = np.column_stack([
        model.predict(X) for model in models
    ])
    return np.mean(predictions, axis=1)


In [32]:
test_y = averagingModels(test, train, y, [GBR, lasso, ENet, m_lgb])
test_y = np.exp(test_y)

In [33]:
test_submit= pd.DataFrame({'Id': testid.Id, 'SalePrice':test_y})
test_submit.shape
test_submit.to_csv('essai10.csv', index=False)